# Observer Pattern Implementation in Python 

In this article we will consider one of the very well known behavioral patterns called the **observer pattern**.

In this pattern we are isolating a group of objects called `Observer` from the `Observable` (also known as `Subject`).

The `Observer` is interested in a state change of the `Observable`.
The role of the `Observable` is to notify all registered observers about the state update (similar to a notification subscription on our phones).

We will implement initially a single-threaded version of this pattern.
Then we will discuss the potential issues in a multi threaded approach and finally progress to a thread safe version that is as performant as we can make it.

### Creating Auxiliary Classes
We will define the auxiliary classes that will be needed.

We require an `Event` object definition that will be used to pass information into the `Observer` from the `Observable`.

In [1]:
from dataclasses import dataclass


@dataclass
class Event:
    name: str
    data: dict

### Defining Interfaces for the Observer Pattern
Below we define the `Observer` and `Observable` as abstract classes with their methods.

In [2]:
from abc import ABC, abstractmethod



class IObserver(ABC):
    @abstractmethod
    def update(self, event: Event, key: str):
        raise NotImplementedError


class IObservable(ABC):
    @abstractmethod
    def add_observer(self, observer: IObserver, key: str):
        raise NotImplementedError
    
    @abstractmethod
    def remove_observer(self, observer: IObserver, key: str):
        raise NotImplementedError
    
    @abstractmethod
    def update_observers(self, event: Event, key: str):
        raise NotImplementedError

### Implementing Concrete Versions for Single Threaded Use Case
Now that we have the necessary building blocks we can implement a single threaded version of the pattern and profile its performance.